# Dataframe Construction Part 2:
Assigning every trap to zip code data using Haversine.

In [1]:
import pandas as pd
from haversine import haversine
import numpy as np

In [2]:
#Import the data.
data1 = pd.read_csv('assets/train_1.csv')
data2 = pd.read_csv('assets/test_1.csv')
data3 = pd.read_csv('assets/SCRAPED_ZIP_CODE_DETAILS.csv')

#Put it into dataframes.
df_train = pd.DataFrame(data1)
df_test = pd.DataFrame(data2)
df_zip = pd.DataFrame(data3)

## Calculate trap distance to each of the 55 zip codes.

In [3]:
#Nathan doing stupid stuff that shouldn't take so many lines of code.
df1 = df_zip.copy().groupby(['Zip Code']).mean().reset_index().transpose().reset_index()
df1.columns = df1.loc[0]
df1.drop([0,1,4,5,6], inplace=True)
df1 = df1.reset_index()
df1.drop(['index', 'Zip Code'], axis=1, inplace=True)

In [4]:
#Grouping the trap data so that this is easier to process.
df_transition = df_train.copy()
df_transition = df_transition.groupby(['Trap']).mean().reset_index()
df_transition.drop(['Unnamed: 0', 'Block', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent', 'station'], axis=1, inplace=True)

In [5]:
#Function to find the distance from every trap to each zip code.
df2 = pd.DataFrame(columns=[df1.columns])
for i in df1.columns:
    holder = []
    for x, y in enumerate(df_train['Latitude']):
        zip_code = (df1[i].loc[0], df1[i].loc[1])
        trap = (y, df_train['Longitude'].loc[x])
        holder.append(haversine(zip_code, trap, miles=True))
    df2[i] = holder

In [6]:
df3 = df2.copy()
df4 = pd.DataFrame(columns=[df3.columns])

In [7]:
#Function to order the zip code distance to each trap.
for i in df3.columns:
    holder2 = []
    for x in df3.index:
        y = df3.loc[x].idxmin()
        holder2.append(y)
        df3[y].loc[x] = np.nan
    df4[i] = holder2

In [11]:
df5 = df4[df4.columns[:3]].copy()
print df5.shape
df5.head()

(10506, 3)


,60601.0,60602.0,60603.0
0,60634.0,60656.0,60630.0
1,60634.0,60656.0,60630.0
2,60646.0,60630.0,60631.0
3,60656.0,60631.0,60634.0
4,60656.0,60631.0,60634.0


In [19]:
#check = (list(df4[60601.0]) + list(df4[60602.0]) + list(df4[60603.0]))
check = list(df4[60601.0])

In [20]:
len(set(check))

46

In [16]:
df6 = df2.copy()
df7 = pd.DataFrame(columns=[df6.columns])

In [18]:
#Function to order the zip code distance to each trap. Same thing as earlier but to get the distances.
for i in df6.columns:
    holder3 = []
    for x in df6.index:
        y = df6.loc[x].min()
        z = df6.loc[x].idxmin()
        holder3.append(y)
        df6[z].loc[x] = np.nan
    df7[i] = holder2

In [29]:
df_train['Zip Code'] = df5[60601.0]

In [36]:
print df_train.shape
df_train.head(2)

(10506, 16)


,Unnamed: 0,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,station,zip,Zip Code
0,0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,60634.0,60634.0
1,1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0,60634.0,60634.0


In [28]:
df_zip.columns

Index([u'Unnamed: 0', u'Zip Code', u'Location', u'City', u'Population',
       u'People / Sq. Mile', u'National Rank', u'Latitude', u'Longitude',
       u'Population.1', u'Areasqmile', u'Landsqmile', u'Watersqmile',
       u'Poppersqmile'],
      dtype='object')

In [38]:
#Join zip code dataframe to df_train on the zip column
df_train2 = pd.merge(df_train, df_zip[['Zip Code', 'Areasqmile', 'Landsqmile', 'Watersqmile', 'Poppersqmile']], how='left', on='Zip Code')

In [40]:
print df_train2.shape
df_train2.head()

(10506, 20)


,Unnamed: 0,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,station,zip,Zip Code,Areasqmile,Landsqmile,Watersqmile,Poppersqmile
0,0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,60634.0,60634.0,6.99,6.98,0.01,"10,626.10"
1,1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,60634.0,60634.0,6.99,6.98,0.01,"10,626.10"
2,2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,0,60646.0,60646.0,4.71,4.71,0.00,"5,741.47"
3,3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,0,60656.0,60656.0,3.57,3.57,0.00,"7,590.97"
4,4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,0,60656.0,60656.0,3.57,3.57,0.00,"7,590.97"


In [ ]:
#assign latitude longitude to each weather station.
df_weather['Latitude'] = df_weather['Station'].map({1:41.995, 2:41.786})
df_weather['Longitude'] = df_weather['Station'].map({1:-87.933, 2:-87.752})
df_weather.head(2)